In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from datetime import datetime

import numpy as np
import pandas as pd
from datetime import datetime
import json
from tqdm import tqdm
from sklearn.preprocessing import MultiLabelBinarizer
import itertools
import scipy as sp
from sklearn.feature_extraction import DictVectorizer
# from pyfm import pylibfm
from sklearn.preprocessing import normalize
from scipy import sparse

from numpy.linalg import norm
from sklearn.pipeline import FeatureUnion
from scipy.sparse import coo_matrix
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

In [ ]:
#Importing Review Data
line_count = len(open("/content/drive/Shared drives/Fall2020DSGirlPower/SI650/project/DATA/yelp_academic_dataset_review.json").readlines())
user_ids, business_ids, stars, dates, text = [], [], [], [], []
with open("/content/drive/Shared drives/Fall2020DSGirlPower/SI650/project/DATA/yelp_academic_dataset_review.json") as f:
  for line in tqdm(f, total=line_count):
       blob = json.loads(line)
       user_ids += [blob["user_id"]]
       business_ids += [blob["business_id"]]
       stars += [blob["stars"]]
       dates += [blob["date"]]
       text += [blob["text"]]
ratings = pd.DataFrame(
   {"user_id": user_ids, "business_id": business_ids, "rating": stars, "date": dates, "text": text}
)
user_counts = ratings["user_id"].value_counts()
active_users = user_counts.loc[user_counts >= 5].index.tolist()

100%|██████████| 8021122/8021122 [01:57<00:00, 68534.63it/s]


In [ ]:
# read business data
bus = pd.read_csv("/content/drive/Shared drives/Fall2020DSGirlPower/SI650/project/DATA/TO_data/business.csv")

In [ ]:
#Importing Business data
line_count = len(open("/content/drive/Shared drives/Fall2020DSGirlPower/SI650/project/DATA/yelp_academic_dataset_business.json").readlines())
business_ids, names, addresses, cities, states, postal_codes, latitudes, longitudes, stars, review_counts, is_open, attributes, categories, hours = [],[],[],[],[],[],[],[],[],[],[],[],[],[]
with open("/content/drive/Shared drives/Fall2020DSGirlPower/SI650/project/DATA/yelp_academic_dataset_business.json") as f:
  for line in tqdm(f, total=line_count):
        blob = json.loads(line)
        business_ids += [blob["business_id"]]
        names += [blob["name"]]
        addresses += [blob["address"]]
        cities += [blob["city"]]
        states += [blob["state"]]
        postal_codes += [blob["postal_code"]]
        latitudes += [blob["latitude"]]
        longitudes += [blob["longitude"]]
        stars += [blob["stars"]]
        review_counts += [blob["review_count"]]
        is_open += [blob["is_open"]]
        attributes += [blob["attributes"]]
        categories += [blob["categories"]]
        hours += [blob["hours"]]
businesses = pd.DataFrame(
    {"business_id": business_ids, "name": names, "address": addresses, "city": cities, "state": states, "postal_code": postal_codes, "latitude": latitudes, "longitude": longitudes, "rating": stars, "review_count": review_counts, "is_open": is_open, "attributes": attributes, "categories": categories, "hours": hours}
)

100%|██████████| 209393/209393 [00:04<00:00, 48031.38it/s]


In [ ]:
#Imporing User data

line_count = len(open("/content/drive/Shared drives/Fall2020DSGirlPower/SI650/project/DATA/yelp_academic_dataset_user.json").readlines())
user_id, review_cnt, yelping_since, useful, funny, cool, fans, avg_user_rating, yrs_elite, \
compliment_hot, compliment_more, compliment_profile, compliment_cute, compliment_list, \
compliment_note, compliment_plain, compliment_cool, \
compliment_funny, compliment_writer, compliment_photos = [], [], [], [],[], [], [], [], [],[], [], [], [], [],[], [], [], [], [],[] 

with open("/content/drive/Shared drives/Fall2020DSGirlPower/SI650/project/DATA/yelp_academic_dataset_user.json") as f:
  for line in tqdm(f, total=line_count):
       blob = json.loads(line)
       user_id += [blob["user_id"]]
       review_cnt += [blob["review_count"]]
       yelping_since += [blob["yelping_since"]]
       useful += [blob["useful"]]
       funny += [blob["funny"]]
       cool += [blob["cool"]]
       fans += [blob["fans"]]
       avg_user_rating += [blob["average_stars"]]
       yrs_elite += [blob["elite"]]
       compliment_hot += [blob['compliment_hot']]
       compliment_more += [blob['compliment_more']]
       compliment_profile += [blob['compliment_profile']]
       compliment_cute += [blob['compliment_cute']]
       compliment_list += [blob['compliment_list']]
       compliment_note += [blob['compliment_note']]
       compliment_plain += [blob['compliment_plain']]
       compliment_funny += [blob['compliment_funny']]
       compliment_writer += [blob['compliment_writer']]
       compliment_photos += [blob['compliment_photos']]
user = pd.DataFrame(
   {"user_id": user_id, "review_cnt": review_cnt, "yelping_since": yelping_since, "useful": useful,"funny": funny,
    "cool": cool, "fans": fans, "avg_user_rating": avg_user_rating, "yrs_elite": yrs_elite, "compliment_hot": compliment_hot,
    "compliment_more": compliment_more, "compliment_profile": compliment_profile, "compliment_cute": compliment_cute,
   "compliment_list": compliment_list, "compliment_note": compliment_note, "compliment_plain": compliment_plain,
   "compliment_funny": compliment_funny, "compliment_writer": compliment_writer, "compliment_photos": compliment_photos}
)

100%|██████████| 1968703/1968703 [01:04<00:00, 30587.48it/s]


### Filter business data

In [ ]:
businesses.head(2)

,business_id,name,address,city,state,postal_code,latitude,longitude,rating,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None


In [ ]:
businesses.shape

(209393, 14)

In [ ]:
businesses["city"].value_counts()

Las Vegas          31631
Toronto            20366
Phoenix            20171
Charlotte          10422
Scottsdale          9342
                   ...  
Ranlo                  1
Peoria,                1
Black Rock City        1
Cranberry Twp          1
Strip District         1
Name: city, Length: 1251, dtype: int64

In [ ]:
bus = businesses[businesses["city"] == "Toronto"]

In [ ]:
bus.shape

(20366, 14)

In [ ]:
from google.colab import files

In [ ]:
bus.to_csv('business.csv')
files.download("business.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### filte review data

In [ ]:
ratings.head(2)

,user_id,business_id,rating,date,text
0,OwjRMXRC0KyPrIlcjaXeFQ,-MhfebM0QIsKt87iDN-FNw,2.0,2015-04-15 05:21:16,"As someone who has worked with many museums, I..."
1,nIJD_7ZXHq-FX8byPMOkMQ,lbrU8StCq3yDfr-QMnGrmQ,1.0,2013-12-07 03:16:52,I am actually horrified this place is still in...


In [ ]:
len(active_users)

343872

In [ ]:
ratings.shape

(8021122, 5)

In [ ]:
len(ratings["user_id"].unique())

1968703

In [ ]:
act_users = set(active_users)

In [ ]:
# get business in Toronto
bus.head(2)

,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,rating,review_count,is_open,attributes,categories,hours
0,9,EosRKXIGeSWFYWwpkbhNnA,Xtreme Couture,700 Kipling Avenue Etobicoke,Toronto,ON,M8Z 5G3,43.624539,-79.529108,3.0,16,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Martial Arts, Gyms, Fitness & Instruction, Act...","{'Monday': '5:30-23:0', 'Tuesday': '5:30-23:0'..."
1,26,1wWneWD_E1pBIyVpdHMaQg,Air Jordan Store,306 Yonge Street,Toronto,ON,M5B 1R4,43.656542,-79.381308,4.0,9,1,"{'RestaurantsPriceRange2': '3', 'BikeParking':...","Shopping, Shoe Stores, Fashion","{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'..."


In [ ]:
bus_unit = set(bus.business_id)	
ra_to = ratings[ratings["business_id"].isin(bus_unit)]

In [ ]:
ra_to.shape

(600584, 5)

In [ ]:
ra_to.to_csv('reviews_TO.csv')

In [ ]:
# export csv file to google drive
!cp reviews_TO.csv "/content/drive/Shared drives/Fall2020DSGirlPower/SI650/project/DATA/TO_data/"

In [ ]:
files.download("reviews_TO.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# filter inactive users
ra_to_active = ra_to[ra_to["user_id"].isin(act_users)]

In [ ]:
ra_to_active.shape

(465376, 5)

In [ ]:
ra_to_active.to_csv('reviews_TO_act.csv')

In [ ]:
# export csv file to google drive
!cp reviews_TO_act.csv "/content/drive/Shared drives/Fall2020DSGirlPower/SI650/project/DATA/TO_data/"

### user data

In [ ]:
user.to_csv('user.csv')

In [ ]:
# export csv file to google drive
!cp user.csv "/content/drive/Shared drives/Fall2020DSGirlPower/SI650/project/DATA/TO_data/"